## Introduction

The goal of the BeeAI project is to make AI agents interoperable, regardless of their underlying implementation. The project consists of two key components:
- **BeeAI Platform**: The platform to easily discover, run, and compose AI agents from any framework.
- **BeeAI Framework**: A production-grade framework for building AI agents in either Python or TypeScript.

Detailed information on BeeAI can be found [here](https://beeai.dev/).

### What's in this notebook?

This notebook demonstrates the BeeAI Requirement agent, a declarative AI agent that combines language models, tools, and execution requirements to create predictable, controlled behavior across different LLMs.

Why Use Requirement Agent?
- Building agents that work reliably across multiple LLMs is difficult. Most agents are tightly tuned to specific models, with rigid prompts that cause models to misinterpret instructions, skip tools, or hallucinate facts.
- RequirementAgent provides a declarative framework for designing agents that strikes a balance between flexibility and control. It allows for agent behavior that is both predictable and adaptable, without the complexity and limitations of more rigid systems.
- The core concept behind the Requirement Agent is that everything is treated as a tool—including data retrieval, web search, reasoning, and generating final answers. This modular structure ensures valid responses with structured outputs and helps eliminate parsing errors.

You can run this notebook on [**Google Colab**](https://colab.research.google.com/). The notebook uses **Ollama** to provide access to a variety of foundation models for remote execution. The notebook will run faster on Colab if you use the free *T4 GPU* option by selecting *Runtime / Change runtime type* in the Colab system menu.

Run the Next Cell to wrap Notebook output.

In [ ]:
from IPython.display import HTML, display


def set_css():
    display(HTML("\n<style>\n pre{\n white-space: pre-wrap;\n}\n</style>\n"))


get_ipython().events.register("pre_run_cell", set_css)

### Install Libraries
We start by installing the required dependencies, installing Ollama and pulling the granite model.

In [ ]:
%pip install -q langchain_community wikipedia requests==2.32.4 beeai-framework

!curl -fsSL https://ollama.com/install.sh | sh > /dev/null
!nohup ollama serve >/dev/null 2>&1 &
!ollama pull granite3.3:8b

### Import Libraries

In [ ]:
# Imports
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from pydantic import BaseModel, Field

from beeai_framework.agents.requirement import RequirementAgent
from beeai_framework.backend.chat import ChatModel
from beeai_framework.context import RunContext
from beeai_framework.emitter.emitter import Emitter
from beeai_framework.memory.unconstrained_memory import UnconstrainedMemory
from beeai_framework.tools import StringToolOutput, Tool, tool
from beeai_framework.tools.types import ToolRunOptions
from beeai_framework.tools.weather.openmeteo import OpenMeteoTool

print("Imports and credentials completed")

## Basic Requirement agent

To configure a Requirement agent, you need to define a ChatModel and construct an Agent.

In this example, we won't provide any external tools to the agent. It will rely solely on its own memory to provide answers. This is a basic setup where the agent tries to reason and act based on the context it has built internally.

Try modifying the input text in the call to agent.run() to experiment with obtaining different answers. This will help you see how the agent's reasoning and response vary with different prompts.

In [ ]:
# Construct ChatModel
chat_model = ChatModel.from_name("ollama:granite3.3:8b")

# Construct Agent instance with the chat model
agent = RequirementAgent(llm=chat_model, tools=[], memory=UnconstrainedMemory())

# Run the agent
message = "What chemical elements make up a water molecule?"
print("Question: " + message + "\n")
response = await agent.run(message)
print(response.last_message.text)

## Provide tools to the agent

To go beyond just chatting with an LLM, you can provide tools to the agent. This enables the agent to perform specific tasks and interact with external systems, enhancing its functionality. There are different ways to add tools to the agent:

- Built-in tools from the framework: BeeAI provides several built-in tools that you can easily integrate with your agent.
- Importing tools from other libraries: You can bring in external tools or APIs to extend your agent's capabilities.
- Custom tooling: You can also write your own custom tools tailored to your specific use case.

By equipping the agent with these tools, you allow it to perform more complex actions, such as querying databases, interacting with APIs, or manipulating data.

### Built-in tools

BeeAI comes with several built-in tools that are part of the library, which can be easily imported and added to your agent.

In this example, we provide the agent with a weather forecast lookup tool called OpenMeteoTool. With this tool, the agent can retrieve real-time weather data, enabling it to answer weather-related queries with more accuracy.

Follow the agent's thoughts and actions to understand how it approaches and solves the problem.

In [ ]:
# create an agent using the default LLM and add the OpenMeteoTool that is capable of fetching weather-based information
agent = RequirementAgent(llm=chat_model, tools=[OpenMeteoTool()], memory=UnconstrainedMemory())

# Run the agent
message = "What's the current weather in London?"
print("Question: " + message + "\n")

response = await agent.run(message)
print(response.last_message.text)

## Imported tools

Tools can also be imported from other libraries to extend the functionality of your agent. For example, you can integrate tools from libraries like LangChain to give your agent access to even more capabilities.

Here’s an example showing how to integrate the Wikipedia tool from LangChain, written in long form (without using the @tool decorator):

In [ ]:
class LangChainWikipediaToolInput(BaseModel):
    query: str = Field(description="The topic or question to search for on Wikipedia.")


class LangChainWikipediaTool(Tool[LangChainWikipediaToolInput, ToolRunOptions, StringToolOutput]):
    """Adapter class to integrate LangChain's Wikipedia tool with our framework"""

    name = "Wikipedia"
    description = "Search factual and historical information from Wikipedia about given topics."
    input_schema = LangChainWikipediaToolInput

    def __init__(self) -> None:
        super().__init__()
        self._wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

    def _create_emitter(self) -> Emitter:
        return Emitter.root().child(
            namespace=["tool", "search", "langchain_wikipedia"],
            creator=self,
        )

    async def _run(
        self, input: LangChainWikipediaToolInput, options: ToolRunOptions | None, context: RunContext
    ) -> StringToolOutput:
        query = input.query
        try:
            result = self._wikipedia.run(query)
            return StringToolOutput(result=result)
        except Exception as e:
            print(f"Wikipedia search error: {e!s}")
            return f"Error searching Wikipedia: {e!s}"


agent = RequirementAgent(llm=chat_model, tools=[LangChainWikipediaTool()], memory=UnconstrainedMemory())
message = "Who is the current president of the European Commission?"
print("Question: " + message + "\n")

response = await agent.run(message)
print(response.last_message.text)

The previous example can be re-written in a shorter form by adding the @tool decorator, which simplifies the tool definition. Here's how you can do it:

In [ ]:
# defining a tool using the `tool` decorator
# Note: the pydoc is important as it serves as the tool description to the agent
@tool
def langchain_wikipedia_tool(query: str) -> str:
    """
    Search factual and historical information, including biography, history, politics, geography, society, culture,
    science, technology, people, animal species, mathematics, and other subjects.

    Args:
        query: The topic or question to search for on Wikipedia.

    Returns:
        The information found via searching Wikipedia.
    """
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    return wikipedia.run(query)


# using the tool in an agent
agent = RequirementAgent(llm=chat_model, tools=[langchain_wikipedia_tool], memory=UnconstrainedMemory())
message = "What is the longest living vertebrate?"
print("Question: " + message + "\n")

response = await agent.run(message)
print(response.last_message.text)

## Learn More

Detailed information on BeeAI can be found [here](https://beeai.dev/).

In this notebook, you learned how to build BeeAI Requirement agents in the BeeAI Framework.